In [1]:
# 0. bs4 모듈의 BeautifulSoup 클래스 사용
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

In [2]:
# 1. 웹(html)문서 불러오기
# Client 요청 -> requests.get('http://URL주소')
#                response = headers + content  <- Server 응답
# response(응답)은 headers + content 으로 구성되어 있다.
# response.headers 는 HTTP 헤더 내용을 사전(dict)으로 조회
# response.content 는 HTTP 바디의 내용을 Binary(그림,압축,문서) 코드로 조회
# response.text    는 HTTP 바디의 내용을 Text(텍스트, HTML문서) 문자로 조회
# response.encoding은 HTTP 바디가 어떤 인코딩 방식으로 만들어졌는지 조회
url = 'http://192.168.50.54/index.html'
response = requests.get(url)
print( response )
print( response.headers )
response.encoding = 'UTF-8'
print( response.encoding )
print( response.text[:69] )

<Response [200]>
{'Content-Type': 'text/html', 'Last-Modified': 'Sun, 03 Mar 2024 05:28:33 GMT', 'Accept-Ranges': 'bytes', 'ETag': '"85844da22b6dda1:0"', 'Server': 'Microsoft-IIS/10.0', 'Date': 'Sat, 09 Mar 2024 00:59:11 GMT', 'Content-Length': '943'}
UTF-8
<html>
	<head>
		<title>테이블의 구조</title>
		<meta charset="UTF-8" />


In [3]:
# 2. 웹문서내용(str,문자열)을 분석 가능한 객체(bs4.BeautifulSoup) 생성
# 3. find, find_all 메소드등을 이용하여 특정한 <tag> 블럭 조회
text = response.text 
html = BeautifulSoup(text, 'html.parser')
tr_list = html.find('tbody').find_all('tr')
rows = []
for tr in tr_list:
    td_list = tr.find_all('td')
    row = []
    for td in td_list:
        data = td.text
        row.append(data)
    rows.append(row)
rows

[['1행_1번째', '1행_2번째', '1행_3번째'],
 ['2행_1번째', '2행_2번째', '2행_3번째'],
 ['3행_1번째', '3행_2번째', '3행_3번째'],
 ['4행_1번째', '4행_2번째', '4행_3번째']]

In [4]:
url = 'http://192.168.50.54/professors.html'
response = requests.get(url)
# response = headers + content(text, jpeg)
print( response.headers )
text = response.text
print( text[:57] )

{'Content-Type': 'text/html', 'Last-Modified': 'Mon, 07 Mar 2022 23:16:00 GMT', 'Accept-Ranges': 'bytes', 'ETag': '"0b8af4e7932d81:0"', 'Server': 'Microsoft-IIS/10.0', 'Date': 'Sat, 09 Mar 2024 00:59:11 GMT', 'Content-Length': '9161'}
<html>
	<head>
		<title>This is jsoup test page</title>


In [19]:
def make_row(tr):
    no = tr.find('td', {'class':'number'}).text
    name = tr.find('td', {'class':'professor'}).text
    lecture = tr.find('td', {'class':'lecture'}).text
    grade = tr.find('td', {'class':'grade'}).text
    evl = tr.find('td', {'class':'evaluation'}).text
    row = [no, name, lecture, grade, evl]
    return row

html = BeautifulSoup(text, 'html.parser')
tr_list = html.find('tbody').find_all('tr')
rows = []
for tr in tr_list:
    row = make_row(tr)
    rows.append(row)
df = pd.DataFrame(rows, columns = ['no', 'name', 'lecture', 'grade', 'evaluation'])
df.set_index('no', inplace = True)
df.to_csv('csv/professors.csv')
df

,name,lecture,grade,evaluation
no,,,,
1,John,C Language,2,1P
2,Kim,JAVA,3,2.5P
3,Gun,PHP / Python,1,5P
4,Hun,Soccer,2,5P
5,Coner,Smart Car,1,4P


In [11]:
tr = tr_list[0]
grade = tr.find('td', {'class':'grade'}).text
evl = tr.find('td', {'class':'evaluation'}).text

'1P'